In [1]:
import json
import numpy as np
import networkx as nx
from scipy.optimize import minimize
from plotter_lib import *

### Definimos la función error para el optimizador

In [2]:
def get_metricas(links, w):

    G = nx.DiGraph()

    for i in range(len(links)):

        n1, n2 = links[i].split(' ')
        G.add_edge(n1, n2, weight=w[i])

    ec = nx.eigenvector_centrality(G, weight="weight")
    c = nx.clustering(G, weight="weight")

    return [ec, c]

alpha =1
beta= 1.075

def fobj(pars, args):

    links, players, mu = args

    Ec, C = get_metricas(links, np.array(pars, dtype=int))


    return sum( alpha*(Ec[player] - mu["ec"][player])**2 + beta*(C[player] - mu["c"][player])**2 for player in players)


### Preparo los argumentos

In [3]:
# jugadores titulares
#players = ['Player1', 'Player2', 'Player3', 'Player4', 'Player5', 'Player6', 'Player7', 'Player8', 'Player9', 'Player10']
players = [ 'Player1', 'Player3', 'Player4', 'Player5', 'Player6', 'Player7', 'Player8', 'Player9', 'Player10']


# todos los posibles links
n = len(players)
links = [ ]

for i in range(n):
    for j in range(n):
        if i!=j:
            links.append( "%s %s"%(players[i],players[j]) )



#elejimos metricas
arx = 'player_metrica_S1.json'
d = json.load(open(arx, "r"))

mu={}
mu["ec"]={  player:np.mean(d[player]["ec"]) for player in players }
mu["c"]={  player:np.mean(d[player]["c"]) for player in players }




args = [ links,players,mu  ]


### Optimizo

In [4]:

pars0 = np.ones(len(links))
bounds = [ (0, 10) for i in range(len(links)) ]

res = minimize(fobj, pars0, args=args, method="Powell", bounds=bounds)

res



 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 0.028969384074697697
       x: [ 7.831e+00  8.068e+00 ...  2.147e+00  2.369e+00]
     nit: 2
   direc: [[ 0.000e+00  0.000e+00 ...  0.000e+00  1.000e+00]
           [ 0.000e+00  1.000e+00 ...  0.000e+00  0.000e+00]
           ...
           [ 0.000e+00  0.000e+00 ...  1.000e+00  0.000e+00]
           [ 1.086e+00  1.392e+00 ...  3.731e-02  1.583e-02]]
    nfev: 3203

### Chequeo que el resultado sea correcto

In [5]:
pesos = res.x.astype(int)

In [6]:
Ec,C = get_metricas(links, res.x)

sd={}
sd["ec"]={  player:np.std(d[player]["ec"]) for player in players }
sd["c"]={  player:np.std(d[player]["c"]) for player in players }
count_no=0

print ("Eigenvector Centrality")
for player in players:

    l1,l2 = mu["ec"][player]-sd["ec"][player], mu["ec"][player]+sd["ec"][player]
    
    if Ec[player]>l2 or Ec[player]<l1:
        count_no+=1
        print ( player, l1, "| %f |" %Ec[player] , l2, "  X")
    else:
        print ( player, l1, "| %f |" %Ec[player] , l2)


print ("\nClustering")
for player in players:

    l1,l2 = mu["c"][player]-sd["c"][player], mu["c"][player]+sd["c"][player]
    
    if C[player]>l2 or C[player]<l1:
        count_no+=1
        print ( player, l1, "| %f |" %C[player] , l2, "  X")
    else:
         print ( player, l1, "| %f |" %C[player] , l2)


print ("\nNro casos fuera:", count_no)

Eigenvector Centrality
Player1 0.1808256798117162 | 0.240755 | 0.3026705423892874
Player3 0.04514041289076494 | 0.222192 | 0.2264852319550637
Player4 0.2812174258093085 | 0.436777 | 0.5469438080445308
Player5 0.3126430869251662 | 0.418971 | 0.45285535628454887
Player6 0.2769308847557014 | 0.375110 | 0.4234938191200025
Player7 0.08542764399209277 | 0.306586 | 0.4388688297596225
Player8 0.22050169323507524 | 0.344596 | 0.4067762901777438
Player9 0.19787303019639885 | 0.370780 | 0.5000898047425932
Player10 0.0731479838757895 | 0.188231 | 0.18845958167778673

Clustering
Player1 0.13394274667139172 | 0.293068 | 0.2422053079314596   X
Player3 0.11177358976557374 | 0.240736 | 0.27668014387187073
Player4 0.08550813310025977 | 0.268990 | 0.24210998488017754   X
Player5 0.0938094907727953 | 0.269156 | 0.1964482583719803   X
Player6 0.10811612510458521 | 0.255052 | 0.24836623090152374   X
Player7 0.09670722426056347 | 0.227827 | 0.22796137416303439
Player8 0.10542207430563524 | 0.226403 | 0.18708

In [7]:
edgelist= [ "%s %d" % (links[i],pesos[i]) for i in range(len(links)) if pesos[i]>0 ]

np.savetxt("red_optima_truncada_p2.edgelist", edgelist, fmt="%s")



In [8]:
#players = ['Player1', 'Player2', 'Player3', 'Player4', 'Player5', 'Player6', 'Player7', 'Player8', 'Player9', 'Player10']

#colorDict = {    player:"C%d"%i for i,player in enumerate(players)}

#json.dump(colorDict, open("colorDict.json","w") )